In [1]:
import torch
from xautodl import spaces
from xautodl.xlayers import super_core

def _create_stel(input_dim, output_dim, order):
    return super_core.SuperSequential(
        super_core.SuperLinear(input_dim, output_dim),
        super_core.SuperTransformerEncoderLayer(
            output_dim,
            num_heads=spaces.Categorical(2, 4, 6),
            mlp_hidden_multiplier=spaces.Categorical(1, 2, 4),
            order=order,
        ),
    )

In [2]:
batch, seq_dim, input_dim = 1, 4, 6
order = super_core.LayerOrder.PreNorm

In [3]:
out1_dim = spaces.Categorical(12, 24, 36)
out2_dim = spaces.Categorical(24, 36, 48)
out3_dim = spaces.Categorical(36, 72, 100)
layer1 = _create_stel(input_dim, out1_dim, order)
layer2 = _create_stel(out1_dim, out2_dim, order)
layer3 = _create_stel(out2_dim, out3_dim, order)
model = super_core.SuperSequential(layer1, layer2, layer3)
print(model)

SuperSequential(
  (0): SuperSequential(
    (0): SuperLinear(in_features=6, out_features=Categorical(candidates=[12, 24, 36], default_index=None), bias=True)
    (1): SuperTransformerEncoderLayer(
      (norm1): SuperLayerNorm1D(shape=Categorical(candidates=[12, 24, 36], default_index=None), eps=1e-06, elementwise_affine=True)
      (mha): SuperSelfAttention(
        input_dim=Categorical(candidates=[12, 24, 36], default_index=None), proj_dim=Categorical(candidates=[12, 24, 36], default_index=None), num_heads=Categorical(candidates=[2, 4, 6], default_index=None), mask=False, infinity=1000000000.0
        (q_fc): SuperLinear(in_features=Categorical(candidates=[12, 24, 36], default_index=None), out_features=Categorical(candidates=[12, 24, 36], default_index=None), bias=False)
        (k_fc): SuperLinear(in_features=Categorical(candidates=[12, 24, 36], default_index=None), out_features=Categorical(candidates=[12, 24, 36], default_index=None), bias=False)
        (v_fc): SuperLinear(in_fe

In [ ]:
inputs = torch.rand(batch, seq_dim, input_dim)
outputs = model(inputs)

> /Users/xuanyidong/anaconda3/lib/python3.8/site-packages/xautodl-0.9.9-py3.8.egg/xautodl/xlayers/super_transformer.py(116)forward_raw()
    114               import pdb; pdb.set_trace()
    115             # feed-forward layer -- MLP
--> 116             y = self.norm2(x)
    117             outs = x + self.mlp(y)
    118         elif self._order is LayerOrder.PostNorm:

ipdb> print(self)
SuperTransformerEncoderLayer(
  (norm1): SuperLayerNorm1D(shape=Categorical(candidates=[36, 72, 100], default_index=None), eps=1e-06, elementwise_affine=True)
  (mha): SuperSelfAttention(
    input_dim=Categorical(candidates=[36, 72, 100], default_index=None), proj_dim=Categorical(candidates=[36, 72, 100], default_index=None), num_heads=Categorical(candidates=[2, 4, 6], default_index=None), mask=False, infinity=1000000000.0
    (q_fc): SuperLinear(in_features=Categorical(candidates=[36, 72, 100], default_index=None), out_features=Categorical(candidates=[36, 72, 100], default_index=None), bias=False)
 

In [ ]:
abstract_space = model.abstract_search_space
abstract_space.clean_last()
abstract_child = abstract_space.random(reuse_last=True)
# print("The abstract child program is:\n{:}".format(abstract_child))
model.enable_candidate()
model.set_super_run_type(super_core.SuperRunMode.Candidate)
model.apply_candidate(abstract_child)
outputs = model(inputs)

In [ ]:
print(outputs.shape)